In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import numpy as np
import os
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
data_dir = 'C:\Aries\Dataset'
dataset = ImageFolder(data_dir+'/train', transform=transform)
dataset_val = ImageFolder(data_dir+'/val', transform=transform)
dataset_test = ImageFolder(data_dir+'/test', transform=transform)

# print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")

# Define the data loaders
batch_size = 16
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

# Define the CNN model
class MyCNN(nn.Module):
    def __init__(self, num_classes):
        super(MyCNN, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding = 1, stride = 1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding = 1, stride = 1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding = 1, stride = 1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding = 1, stride = 1)
        
        # Define the fully connected layers
        self.fc1 = nn.Linear(in_features=256 * 16 * 16, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=6)
        
        # Define the activation function
        self.relu = nn.ReLU()
        
        # Define the max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride = 2)
        
    def forward(self, x):
        # Perform the forward pass through the convolutional layers
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Flatten the output from the convolutional layers
        x = x.view(-1, 256 * 16 * 16)
        
        # Perform the forward pass through the fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x

# Initialize the model and the optimizer
model = MyCNN(num_classes=6)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [23]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    
    # Initialize variables for tracking loss and accuracy
    train_loss = 0.0
    train_correct = 0
    
    # Iterate over the training data
    for images, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the training loss and accuracy
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
    
    # Compute the average training loss and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)
    
    # Set the model to evaluation mode
    model.eval()
    
    # Initialize variables for tracking loss and accuracy
    val_loss = 0.0
    val_correct = 0
    
    # Iterate over the validation data
    for images, labels in val_loader:
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs,labels)
        
        # Update the validation loss and accuracy
        val_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        val_correct += (predicted == labels).sum().item()
    
    # Compute the average validation loss and accuracy
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)
    
    # Print the training and validation loss and accuracy
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f},\
        Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Save the weights to Google Drive
    torch.save(model.state_dict(), 'my_model1_final.pth')
    file_name = f'weights_final_epoch_{epoch}.pth'
    save_path = 'C:\Aries\Weights' + file_name
    torch.save(model.state_dict(), save_path)
    
    # Saving the Model
    model_path = 'C:\Aries\my_model2_final.pth'
    torch.save(model.state_dict(), model_path)
    

Epoch [1/10], Train Loss: 0.8937,        Train Acc: 0.6429, Val Loss: 0.7521, Val Acc: 0.6878
Epoch [2/10], Train Loss: 0.6225,        Train Acc: 0.7613, Val Loss: 0.6148, Val Acc: 0.7630
Epoch [3/10], Train Loss: 0.5143,        Train Acc: 0.8077, Val Loss: 0.6734, Val Acc: 0.7643
Epoch [4/10], Train Loss: 0.3943,        Train Acc: 0.8591, Val Loss: 0.6593, Val Acc: 0.7812
Epoch [5/10], Train Loss: 0.2244,        Train Acc: 0.9204, Val Loss: 0.7884, Val Acc: 0.7850
Epoch [6/10], Train Loss: 0.1355,        Train Acc: 0.9550, Val Loss: 1.0219, Val Acc: 0.7793
Epoch [7/10], Train Loss: 0.1165,        Train Acc: 0.9625, Val Loss: 1.0254, Val Acc: 0.7900
Epoch [8/10], Train Loss: 0.0445,        Train Acc: 0.9849, Val Loss: 1.4399, Val Acc: 0.7875
Epoch [9/10], Train Loss: 0.0516,        Train Acc: 0.9820, Val Loss: 1.4595, Val Acc: 0.7699
Epoch [10/10], Train Loss: 0.0739,        Train Acc: 0.9796, Val Loss: 1.5040, Val Acc: 0.7705


In [25]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import numpy as np
# import cv2
import os

# Define the path to the data directory
data_dir = 'C:\Aries\Dataset'

batch_size = 32

# Initialize the model
model = MyCNN(num_classes = 6)

# Load the trained weights into the model
model.load_state_dict(torch.load("C:\Aries\Weightsweights_final_epoch_9.pth"))

# Set the model to evaluation mode
model.eval()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Initialize variables for tracking loss and accuracy
test_loss = 0.0
test_correct = 0

# Iterate over the test data
for images, labels in test_loader:
    # Forward pass
    outputs = model(images)
        
    # Compute the loss
    loss = criterion(outputs, labels)
        
    # Update the test loss and accuracy
    test_loss += loss.item() * images.size(0)
    _, predicted = torch.max(outputs.data, 1)
    test_correct += (predicted == labels).sum().item()

# Compute the average test loss and accuracy
test_loss = test_loss / len(test_loader.dataset)
test_acc = test_correct / len(test_loader.dataset)

# Print the test loss and accuracy
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 1.4336, Test Accuracy: 0.7732


In [32]:
from PIL import Image

filename = '1.jpg'
image = Image.open(filename)


transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

image = transform(image)

# Add a batch dimension to the image
image = image.unsqueeze(0)
model = MyCNN(num_classes = 6)

# Load the trained weights into the model
model.load_state_dict(torch.load("C:\Aries\Weightsweights_final_epoch_7.pth"))
# Get the predicted class probabilities
with torch.no_grad():
    outputs = model(image)

# Convert the predicted class probabilities to class labels
_, predicted = torch.max(outputs.data, 1)

# Print the predicted class label
print("Predicted class label:", classes[predicted.item()])

Predicted class label: Your Name
